In [1]:
import numpy as np
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.svm import SVC

In [2]:
np.random.seed(0)
data = np.random.rand(800, 2)

In [3]:
K = rbf_kernel(data, data, 1.8)

In [4]:
# make some fake data X and y
np.random.seed(0)
train_X = np.random.rand(100, 3)
train_y = np.random.randint(0,10, size=100)
# print(train_y.min(), train_y.max())
svc = SVC()
svc.fit(train_X, train_y)

SVC()

In [5]:
test_X = np.random.rand(1, 3)
svc.predict(test_X)

array([2])

In [27]:
def halving(K, m, candidate_index=None, lambda_=0.001):
    
    n = K.shape[0]
    print(f'number of data: {n}')

    m = min(n, m)
    print(f'number of samples: {m}')

    if candidate_index is None:
        candidate_index = list(range(n))
    print(f'candidate_index: {len(candidate_index)}')

    q = len(candidate_index)

    index = np.zeros(m)
    print(f'number of index: {index.shape}')

    for i in range(m):
        score = np.zeros(n)
        for j in range(q):
            k = candidate_index[j]
            score[j] = np.dot(K[k, :], K[:, k]) / (K[k, k] + lambda_)
        
        I = score.argmax()
        index[i] = candidate_index[I]

        # update K
        K = K - np.dot(K[:, int(index[i])], K[int(index[i]), :]) / (K[int(index[i]), int(index[i])] + lambda_)

    return index

In [28]:
id = halving(K, 400)

number of data: 800
number of samples: 400
candidate_index: 800
number of index: (400,)
ipykernel_launcher:22: RuntimeWarning: invalid value encountered in double_scalars
ipykernel_launcher:29: RuntimeWarning: invalid value encountered in double_scalars
